# Trying to reproduce the trigger requirements by hand

Two alternatives: 
    - Using RDataFrame (what is used in the usual 2D plots)
    - Using loops (quite slow but more visual)

In [1]:
from analysis_tools import Dataset, Process, Feature
from pprint import pprint
import ROOT
ROOT.ROOT.EnableImplicitMT()


# CONSTANTS
DO_LOOPS = False
xx = 34
yy = 26
zz = 64

ADD_TO_JETS = 10
MIN_JETS = 2
MAX_JETS = 999
ADD_TO_LEADING_TAU = 18
ADD_TO_SUBLEADING_TAU = 8
ADD_TO_TRIGGER_JETS = 10

Welcome to JupyROOT 6.18/04


In [2]:
# Declare process and dataset objects
ggf_sm_process = Process("ggf_sm", "GGFSM", color=(0, 0, 0))
ggf_bis = Dataset("ggf_sm_bis",
             "/eos/user/j/jleonhol/HH/htautau_ggf/",
             ggf_sm_process)

# build RDataFrame and open TFile to obtain the results using both methods
files = ggf_bis.get_files()[0:10]
print files
dataframe_files = ROOT.vector(str)()
for f in files:
    dataframe_files.push_back(f)
df = ROOT.RDataFrame("Events", dataframe_files)


if DO_LOOPS:
    f = ROOT.TChain("Events")
    for file in files:
        f.Add(file)
    #f = ROOT.TFile.Open(files[0])


['/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_1.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_10.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_100.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_101.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_102.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_103.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_104.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_105.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_106.root', '/eos/user/j/jleonhol/HH/htautau_ggf//NANO_NANO_107.root']


## RDataFrame

### Previous definitions

In [3]:
# some definitions needed 
def add_to_root(root):
    root.gInterpreter.Declare("""
        using Vfloat = const ROOT::RVec<float>&;      
        ROOT::RVec<ROOT::RVec<float>> lead_sublead(Vfloat pt, Vfloat eta, Vfloat phi, Vfloat mass){
            ROOT::RVec<float> leading_pts = {-1., -1.};
            ROOT::RVec<float> leading_etas = {-1., -1.};
            ROOT::RVec<float> leading_phis = {-1., -1.};
            ROOT::RVec<float> leading_mass = {-1., -1.};
            for (size_t i = 0; i < pt.size(); i++) {
                if (pt[i] > leading_pts[0]){
                    leading_pts[1] = leading_pts[0];
                    leading_etas[1] = leading_etas[0];
                    leading_phis[1] = leading_phis[0];
                    leading_mass[1] = leading_mass[0];
    
                    leading_pts[0] = pt[i];
                    leading_etas[0] = eta[i];
                    leading_phis[0] = phi[i];
                    leading_mass[0] = mass[i];
                } 
                else if (pt[i] > leading_pts[1]){
                    leading_pts[1] = pt[i];
                    leading_etas[1] = eta[i];
                    leading_phis[1] = phi[i];
                    leading_mass[1] = mass[i];
                }
            }
            return ROOT::RVec({leading_pts, leading_etas, leading_phis, leading_mass});
        }
    """)
    
    root.gInterpreter.Declare("""
        Double_t Phi_mpi_pi(Double_t x) {
            while (x >= 3.14159) x -= (2 * 3.14159);
            while (x < -3.14159) x += (2 * 3.14159);
            return x;
        }
    """)

    root.gInterpreter.Declare("""
        #include "TMath.h"
        using Vfloat = const ROOT::RVec<float>&;      
        ROOT::RVec<bool> maskDeltaR(Vfloat eta1, Vfloat phi1, Vfloat eta2, Vfloat phi2, float th_dr) {
            ROOT::RVec<bool> mask;
            for (size_t i = 0; i < eta1.size(); i++){
                bool bigger_deltar = true;
                for (size_t j = 0; j < eta2.size(); j++){
                    Double_t deta = eta1[i] - eta2[j];
                    Double_t dphi = Phi_mpi_pi(phi1[i] - phi2[j]);
                    Double_t dr = TMath::Sqrt(deta * deta + dphi * dphi);
                    if (dr < th_dr) bigger_deltar = false;
                }
                mask.push_back(bigger_deltar);
            }
            return mask;
        }
    """)
    return root
ROOT = add_to_root(ROOT)

In [4]:
df = df.Define("lead_sublead_goodtau_pt",
    "lead_sublead("
        "Tau_pt[abs(Tau_eta) <= 2.1], "
        "Tau_eta[abs(Tau_eta) <= 2.1], "
        "Tau_phi[abs(Tau_eta) <= 2.1], "
        "Tau_mass[abs(Tau_eta) <= 2.1]"
    ")[0]").Define("lead_sublead_goodtau_eta", 
    "lead_sublead("
        "Tau_pt[abs(Tau_eta) <= 2.1], "
        "Tau_eta[abs(Tau_eta) <= 2.1], "
        "Tau_phi[abs(Tau_eta) <= 2.1], "
        "Tau_mass[abs(Tau_eta) <= 2.1]"
    ")[1]").Define("lead_sublead_goodtau_phi", 
    "lead_sublead("
        "Tau_pt[abs(Tau_eta) <= 2.1], "
        "Tau_eta[abs(Tau_eta) <= 2.1], "
        "Tau_phi[abs(Tau_eta) <= 2.1], "
        "Tau_mass[abs(Tau_eta) <= 2.1]"
    ")[2]").Define("lead_sublead_goodtau_mass", 
    "lead_sublead("
        "Tau_pt[abs(Tau_eta) <= 2.1], "
        "Tau_eta[abs(Tau_eta) <= 2.1], "
        "Tau_phi[abs(Tau_eta) <= 2.1], "
        "Tau_mass[abs(Tau_eta) <= 2.1]"
    ")[3]")

In [5]:
df = df.Define("lead_sublead_goodl1tau_pt",
    "lead_sublead("
        "L1Obj_pt[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 2.1], "
        "L1Obj_eta[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 2.1], "
        "L1Obj_phi[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 2.1], "
        "L1Obj_pt[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 2.1]"
    ")[0]").Define("lead_sublead_goodl1tau_eta", 
    "lead_sublead("
        "L1Obj_pt[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 2.1], "
        "L1Obj_eta[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 2.1], "
        "L1Obj_phi[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 2.1], "
        "L1Obj_pt[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 2.1]"
    ")[1]").Define("lead_sublead_goodl1tau_phi", 
    "lead_sublead("
        "L1Obj_pt[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 4.7], "
        "L1Obj_eta[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 4.7], "
        "L1Obj_phi[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 4.7], "
        "L1Obj_pt[L1Obj_type == 1 && L1Obj_iso == 1 && abs(L1Obj_eta) <= 4.7]"
    ")[2]")

### Filtering the number of jets in the category

In [6]:
new_df = df.Filter(
    # ask for a minimum number of jets (depending on the category)
    # with pt > 20 (+ sth depending on the dataset)
    "Jet_pt["
        "Jet_pt >= (20 + {0}) && abs(Jet_eta) <= 4.7 && Jet_jetId >= 2 "
        "&& ((Jet_puId >= 4 && Jet_pt <= 50) || (Jet_pt > 50))"
        "&& maskDeltaR("
            "Jet_eta, "
            "Jet_phi, "
            "lead_sublead_goodtau_eta, "
            "lead_sublead_goodtau_phi, "
            "0.5)"
        "].size() >= {1} "
    # ask for a maximum number of jets (depending on the category)
    # with pt > 20 (+ sth depending on the dataset)
    "&& Jet_pt["
        "Jet_pt >= (20 + {0}) && abs(Jet_eta) <= 4.7 && Jet_jetId >= 2 "
        "&& ((Jet_puId >= 4 && Jet_pt <= 50) || (Jet_pt > 50))"
        "&& maskDeltaR("
            "Jet_eta, "
            "Jet_phi, "
            "lead_sublead_goodtau_eta, "
            "lead_sublead_goodtau_phi, "
            "0.5)"
        "].size() <= {2}".format(
            ADD_TO_JETS,
            MIN_JETS,
            MAX_JETS,
        )
)

### Trigger requirements on the L1 Taus and Jets

In [7]:
name = "DoubleIsoTau32er2p1".format(xx)
new_df = new_df.Define(name, 
    "L1Obj_pt["
        "L1Obj_type == 1 "
        "&& abs(L1Obj_eta) <= 2.1 "
        "&& L1Obj_iso == 1 "
        "&& L1Obj_pt >= 32"
    "].size() >= 2"
)
name = "DoubleIsoTau{}er2p1".format(xx)
new_df = new_df.Define(name, 
    "L1Obj_pt["
        "L1Obj_type == 1 "
        "&& abs(L1Obj_eta) <= 2.1 "
        "&& L1Obj_iso == 1 "
        "&& L1Obj_pt >= {}"
    "].size() >= 2".format(xx)
)
name = "DoubleIsoTau{}er2p1Jet{}dR0p5".format(yy, zz)
new_df = new_df.Define(name,
    "lead_sublead_goodl1tau_pt[0] >= {0} "
    "&& lead_sublead_goodl1tau_pt[1] >= {0}"
    "&& (L1Obj_pt["
        "L1Obj_type == 0 "
        "&& L1Obj_pt >= {1} "
        "&& maskDeltaR("
            "L1Obj_eta, "
            "L1Obj_phi, "
            "lead_sublead_goodl1tau_eta, "
            "lead_sublead_goodl1tau_phi, "
            "0.5)"
        "].size() >= 1)".format(yy, zz))

### Trigger requirements on the Reco Taus and Jets

In [8]:
name = "RecoDoubleIsoTau{}er2p1".format(xx)
new_df = new_df.Define(name, 
    "lead_sublead_goodtau_pt[0] >= ({0} + {1}) "
    "&& lead_sublead_goodtau_pt[1] >= ({0} + {2})".format(
        xx, 
        ADD_TO_LEADING_TAU,
        ADD_TO_SUBLEADING_TAU,
    )
)
name = "RecoDoubleIsoTau{}er2p1".format(32)
new_df = new_df.Define(name, 
    "lead_sublead_goodtau_pt[0] >= ({0} + {1}) "
    "&& lead_sublead_goodtau_pt[1] >= ({0} + {2})".format(
        32, 
        ADD_TO_LEADING_TAU,
        ADD_TO_SUBLEADING_TAU,
    )
)
name = "RecoDoubleIsoTau{}er2p1Jet{}dR0p5".format(yy, zz)
new_df = new_df.Define(name,
    # ask that the two taus have pt greater than yy (+ sth depending on the dataset)
    "lead_sublead_goodtau_pt[0] >= ({0} + {1}) "
    "&& lead_sublead_goodtau_pt[1] >= ({0} + {2})"
    # regarding the jets, we require the offline reqs from the analysis +
    # they are not matched to the leading and subleading taus we selected before
    # first, ask for at least 1 jet with pt > zz (+ sth depending on the dataset)
    "&& (Jet_pt["
        "(Jet_pt >= ({3} + {4})) && abs(Jet_eta) <= 4.7 && Jet_jetId >= 2 "
        "&& ((Jet_puId >= 4 && Jet_pt <= 50) || (Jet_pt > 50))"
        "&& maskDeltaR("
            "Jet_eta, "
            "Jet_phi, "
            "lead_sublead_goodtau_eta, "
            "lead_sublead_goodtau_phi, "
            "0.5)"
        "].size() >= 1)".format(
            yy, 
            ADD_TO_LEADING_TAU,
            ADD_TO_SUBLEADING_TAU,
            zz,
            ADD_TO_TRIGGER_JETS
        )
)

In [9]:
hmodel = ("histo", "", 2, 0, 2)
histo32 = new_df.Define("pass", "DoubleIsoTau32er2p1").Histo1D(hmodel, "pass")
histoxx = new_df.Define("pass", "DoubleIsoTau34er2p1").Histo1D(hmodel, "pass")
histoReco32 = new_df.Define("pass", "RecoDoubleIsoTau32er2p1").Histo1D(hmodel, "pass")
histoRecoxx = new_df.Define("pass", "RecoDoubleIsoTau34er2p1").Histo1D(hmodel, "pass")
histoAND32 = new_df.Define("pass", "DoubleIsoTau32er2p1 && RecoDoubleIsoTau32er2p1").Histo1D(hmodel, "pass")
histoANDxx = new_df.Define("pass", "DoubleIsoTau34er2p1 && RecoDoubleIsoTau34er2p1").Histo1D(hmodel, "pass")

histoyyzz = new_df.Define("pass", "DoubleIsoTau26er2p1Jet64dR0p5").Histo1D(hmodel, "pass")
histoRecoyyzz = new_df.Define("pass", "RecoDoubleIsoTau26er2p1Jet64dR0p5").Histo1D(hmodel, "pass")
histoANDyyzz = new_df.Define("pass", "DoubleIsoTau26er2p1Jet64dR0p5 && RecoDoubleIsoTau26er2p1Jet64dR0p5").Histo1D(hmodel, "pass")

histoORxxyyzz = new_df.Define("pass",
                              "(DoubleIsoTau34er2p1 && RecoDoubleIsoTau34er2p1)"
                              "|| (DoubleIsoTau26er2p1Jet64dR0p5 && RecoDoubleIsoTau26er2p1Jet64dR0p5)").Histo1D(hmodel, "pass")

## Using loops

### Functions needed to compute deltaR(tau, jet)

In [10]:
def Phi_mpi_pi(x):
    PI = 3.14159
    while x >= PI: x -= 2 * PI;
    while x < -PI: x += 2 * PI;
    return x;
def get_delta_r(eta1, phi1, eta2, phi2):
    import math
    deta = eta1 - eta2
    dphi = Phi_mpi_pi(phi1 - phi2)
    return math.sqrt(deta * deta + dphi * dphi)

### Actual loop

In [11]:
if DO_LOOPS:

    total_events = 0

    l1_events_xx = 0
    reco_events_xx = 0
    and_events_xx = 0

    l1_events_yy_zz = 0
    reco_events_yy_zz = 0
    and_events_yy_zz = 0

    or_events_xx_yy_zz = 0

    # for event in f.Events:  # to run on a TFile
    for event in f:  # to run on a TChain

        total_events += 1

        ### DoubleIsoTauXXer2p1

        # start storing the 4-vectors for the leading and subleading taus
        # these are used only in the future to compute the deltaR between the taus and jets
        # so we only need eta and phi. pt and mass are stored just to build the TLorentzVector
        # note: as L1 does not compute the object's mass, we fill its spot with the pt as a dummy value, 
        # as it won't be needed
        leading_l1_tau = [-1., -1., -1., -1.]
        subleading_l1_tau = [-1., -1., -1., -1.]
        for L1Obj in zip(event.L1Obj_pt, event.L1Obj_type, event.L1Obj_eta, event.L1Obj_phi, event.L1Obj_iso):
            if L1Obj[1] != 1: continue # skip if it's not a tau
            if abs(L1Obj[2]) > 2.1: continue # skip if the L1 tau doesn't pass the eta req
            if L1Obj[4] != 1: continue # skip if the L1 tau is not isolated

            if L1Obj[0] > leading_l1_tau[0]:
                subleading_l1_tau = [leading_l1_tau[0], leading_l1_tau[1], leading_l1_tau[2], leading_l1_tau[3]]
                leading_l1_tau = [L1Obj[0], L1Obj[2], L1Obj[3], L1Obj[0]]
            elif L1Obj[0] > subleading_l1_tau[0]:
                subleading_l1_tau = [L1Obj[0], L1Obj[2], L1Obj[3], L1Obj[0]]

        leading_tau = [-1., -1., -1., -1.]
        subleading_tau = [-1., -1., -1., -1.]
        for tau in zip(event.Tau_pt, event.Tau_eta, event.Tau_phi, event.Tau_mass):
            if abs(tau[1]) > 2.1: continue
            if tau[0] > leading_tau[0]:
                subleading_tau = [leading_tau[0], leading_tau[1], leading_tau[2], leading_tau[3]]
                leading_tau = [tau[0], tau[1], tau[2], tau[3]]
            elif tau[0] > subleading_tau[0]:
                subleading_tau = [tau[0], tau[1], tau[2], tau[3]]

        # with the leading and subleading taus we can already say if the event passes
        # the trigger reqs for L1, Reco and L1 AND Reco

        l1_xx = (leading_l1_tau[0] >= xx and subleading_l1_tau[0] >= xx)
        reco_xx = (leading_tau[0] >= xx + ADD_TO_LEADING_TAU and subleading_tau[0] >= xx + ADD_TO_SUBLEADING_TAU)

        ### DoubleIsoTauYYer2p1JetZZdR0p5

        # loop over the l1 jets
        triggerl1jets = 0
        jet_pts = []
        for L1Obj in zip(event.L1Obj_pt, event.L1Obj_type, event.L1Obj_eta, event.L1Obj_phi):
            if L1Obj[1] != 0: continue # skip if it's not a jet
            # require no matching between the jet and the selected taus
            if get_delta_r(L1Obj[2], L1Obj[3], leading_l1_tau[1], leading_l1_tau[2]) < 0.5: continue
            if get_delta_r(L1Obj[2], L1Obj[3], subleading_l1_tau[1], subleading_l1_tau[2]) < 0.5: continue

            if L1Obj[0] < zz: continue # skip if its pt is smaller than zz

            triggerl1jets += 1

        # loop over the reco jets
        eventjets = 0
        triggerjets = 0
        for jet in zip(event.Jet_pt, event.Jet_eta, event.Jet_phi, event.Jet_mass, event.Jet_puId, event.Jet_jetId):
            if abs(jet[1]) > 4.7: continue # eta requirement
            if jet[5] < 2: continue # id requirement
            if jet[0] <= 50 and jet[4] < 4: continue # PUiD requirement
            # require no matching between the jet and the selected taus
            if get_delta_r(jet[1], jet[2], leading_tau[1], leading_tau[2]) < 0.5: continue
            if get_delta_r(jet[1], jet[2], subleading_tau[1], subleading_tau[2]) < 0.5: continue
            if jet[0] >= zz + ADD_TO_TRIGGER_JETS:
                triggerjets += 1
            # count also the total jets in the event in order to categorize it
            if jet[0] >= 20 + ADD_TO_JETS:
                eventjets += 1

        l1_yy_zz = ((triggerl1jets >= 1) and (leading_l1_tau[0] >= yy) and (subleading_l1_tau[0] >= yy))
        reco_yy_zz = ((triggerjets >= 1)
            and (leading_tau[0] >= yy + ADD_TO_LEADING_TAU) and (subleading_tau[0] >= yy + ADD_TO_SUBLEADING_TAU))

        # categorization
        if eventjets < MIN_JETS or eventjets > MAX_JETS: continue # looking only at the 2 jet category

        # triggers that were passed
        l1_events_xx += l1_xx
        reco_events_xx += reco_xx
        and_events_xx += (l1_xx and reco_xx)

        l1_events_yy_zz += l1_yy_zz    
        reco_events_yy_zz += reco_yy_zz
        and_events_yy_zz += (l1_yy_zz and reco_yy_zz)

        or_events_xx_yy_zz += ((l1_xx and reco_xx) or (l1_yy_zz and reco_yy_zz))

## Summary

In [12]:
if DO_LOOPS: print "total_events", total_events
print
print "**** RDataFrame ****"
print "l1_events_xx", histoxx.Clone().GetBinContent(2)
print "reco_events_xx", histoRecoxx.Clone().GetBinContent(2)
print "and_events_xx", histoANDxx.Clone().GetBinContent(2)
print
print "l1_events_yy_zz", histoyyzz.Clone().GetBinContent(2)
print "reco_events_yy_zz", histoRecoyyzz.Clone().GetBinContent(2)
print "and_events_yy_zz", histoANDyyzz.Clone().GetBinContent(2)
print
print "or_events_xx_yy_zz", histoORxxyyzz.Clone().GetBinContent(2)
print
print "DoubleIsoTau32"
print "DoubleIsoTau32, L1", histo32.Clone().GetBinContent(2)
print "DoubleIsoTau32, Reco", histoReco32.Clone().GetBinContent(2)
print "DoubleIsoTau32, AND", histoAND32.Clone().GetBinContent(2)
print
print "or_events_xx_yy_zz / DoubleIsoTau32, AND", histoORxxyyzz.Clone().GetBinContent(2) / histoAND32.Clone().GetBinContent(2)
print

if DO_LOOPS:
    print "**** LOOPS ****"
    print "l1_events_xx", l1_events_xx
    print "reco_events_xx", reco_events_xx
    print "and_events_xx", and_events_xx
    print
    print "l1_events_yy_zz", l1_events_yy_zz
    print "reco_events_yy_zz", reco_events_yy_zz
    print "and_events_yy_zz", and_events_yy_zz
    print
    print "or_events_xx_yy_zz", or_events_xx_yy_zz



**** RDataFrame ****
l1_events_xx 25523.0
reco_events_xx 4477.0
and_events_xx 3774.0

l1_events_yy_zz 17753.0
reco_events_yy_zz 5469.0
and_events_yy_zz 4134.0

or_events_xx_yy_zz 5201.0

DoubleIsoTau32
DoubleIsoTau32, L1 28454.0
DoubleIsoTau32, Reco 5217.0
DoubleIsoTau32, AND 4360.0

or_events_xx_yy_zz / DoubleIsoTau32, AND 1.19288990826

